#### Idea Validation Dataset Generation


In [19]:
# imports

import os
import sys

import numpy as np

import open3d as o3d
sys.path.append("..")
import open3d_tutorial as o3dtut
o3dtut.interactive = not "CI" in os.environ
import open3d_trajectory
from open3d_trajectory import CameraPose


In [23]:
%%time

# dataset
dir_dataset_root = "/mnt/d/ICCV2017/"
name_scene = "apartment"
dir_dataset = os.path.join(dir_dataset_root, name_scene)

dir_color = os.path.join(dir_dataset, "image")
dir_depth = os.path.join(dir_dataset, "depth")
list_images_color = [os.path.join(dir_color, color) for color in os.listdir(dir_color)]
list_images_depth = [os.path.join(dir_depth, depth) for depth in os.listdir(dir_depth)]

path_pose = os.path.join(dir_dataset, f"{name_scene}.log")
list_poses = open3d_trajectory.read_trajectory(path_pose)
print(list_poses[1])

[[ 0.99998404  0.00347606  0.00445398  1.99967872]
 [-0.00346543  0.99999113 -0.00239271  2.0013197 ]
 [-0.00446226  0.00237724  0.99998722 -0.29999364]
 [ 0.          0.          0.          1.        ]]
CPU times: user 510 ms, sys: 0 ns, total: 510 ms
Wall time: 704 ms


In [24]:
# unproject from image

color = o3d.io.read_image(list_images_color[0])
depth = o3d.io.read_image(list_images_depth[0])
rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(
    color, depth
)

pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
    rgbd, o3d.camera.PinholeCameraIntrinsic(
        o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault
    )
)

# o3d.visualization.draw_geometries([pcd], zoom=0.5) # FIXME
print(np.asarray(pcd.points))


[[-0.69289809 -0.49963333  1.153     ]
 [-0.6907019  -0.49963333  1.153     ]
 [-0.68850571 -0.49963333  1.153     ]
 ...
 [ 0.69084288  0.57134573  1.30700004]
 [ 0.6933324   0.57134573  1.30700004]
 [ 0.69582193  0.57134573  1.30700004]]


In [32]:
# transform
pcd_world = pcd.transform(list_poses[1].pose)
print(np.asarray(pcd_world.points))

[[ 7.35286847  7.46671946 -0.06464212]
 [ 7.3550641   7.46668916 -0.06468143]
 [ 7.35725973  7.46665886 -0.06472073]
 ...
 [ 8.75394992  8.51696756  0.07464509]
 [ 8.7564388   8.51693321  0.07460054]
 [ 8.75892769  8.51689887  0.07455598]]


In [ ]:
# validate: find transformation from `pcd` ~ `pcd_world`
# NOTE: docker image required?
